### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [24]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [27]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 0 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'save_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False
}

### Do fit

In [28]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(bootstrapper.make_plots(
        show_error_ellipses=p_dict['show_error_ellipses'], show_bootstrap_histograms=p_dict['show_bs_histograms']))
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...


../fitter\fitter.py:310: RuntimeWarning: invalid value encountered in sqrt
  mu = np.sqrt(lam2_chi)


100% complete Time (s):  16.8870000839
Compiling results...
Done.

Total time (s):  18.9010000229 



In [264]:
#print bootstrapper.fits[0]

In [214]:
#bootstrapper.fits[0].show_plots()